We have applied cross validation using different techniques:
- **LogisticRegressionCV**
- **Custom cross validation**
- **cross_val_score**
- **GridSearchCV**
- **RandomSearchCV**

In all cases the evaluated model (Logistic Regression) has a not very big variance (the score is very similar in the different folds)

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
data_train = pd.read_csv('../input/train.csv')

In [3]:
data_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
data_train.shape

(200000, 202)

In [5]:
data_train.isnull().sum().any()

False

In [6]:
data_train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [7]:
X = data_train.drop(['ID_code', 'target'], axis=1)
y = data_train['target'].values

In [8]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier

Let's check if pca can help:

In [9]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(0.99, random_state=1)
pca.fit(X_scaled)

PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=1,
  svd_solver='auto', tol=0.0, whiten=False)

In [10]:
pca.explained_variance_ratio_[:198].sum()

0.9915528132160373

We need a lot of features to explain most of the variance, so pca looks like no very useful here:

In [11]:
pca.n_components_

198

We are going to split our data into a training and testing sets:

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, 
                                                    y, 
                                                    test_size=0.1, 
                                                    stratify=y, 
                                                    random_state=1)

## Cross validation with LogisticRegressionCV
This method is specific for LogisticRegression, although other models have also their own "CV" version. This method let's explore differnt values of hyperparameters at the same time.

In [13]:
kfolds = StratifiedShuffleSplit(n_splits=5, random_state=1)

In [14]:
model_lr = LogisticRegressionCV(Cs=10, cv=kfolds, random_state=1, class_weight='balanced', scoring='roc_auc')

In [15]:
model_lr.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight='balanced',
           cv=StratifiedShuffleSplit(n_splits=5, random_state=1, test_size='default',
            train_size=None),
           dual=False, fit_intercept=True, intercept_scaling=1.0,
           max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
           random_state=1, refit=True, scoring='roc_auc', solver='lbfgs',
           tol=0.0001, verbose=0)

In [16]:
model_lr.score(X_test, y_test)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1926: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.8526643049344715

In [17]:
model_lr.C_

array([0.0001])

## Custom cross validation
At the same time that we create each model in the cross validation process we evaluate it on the validation data and create predictions for the testing set. We will ensamble those predictions and we'll evaluate them.

In [18]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

#model_rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=1)
model_lr = LogisticRegression(C=0.0001, solver='lbfgs', random_state=1)
model = model_lr
scores = []
scores_xtest = []
preds_xtest = np.empty((X_test.shape[0], 5))
for i, (id_train, id_val) in tqdm(enumerate(kfolds.split(X_train, y_train))):
    model.fit(X_train[id_train], y_train[id_train])
    pred_i = model.predict_proba(X_train[id_val])[:, 1]
    score_i = roc_auc_score(y_train[id_val], pred_i)
    scores.append(score_i)
    pred_xtest = model.predict_proba(X_test)[:, 1]
    preds_xtest[:, i] = pred_xtest
    score_xtest = roc_auc_score(y_test, pred_xtest)
    scores_xtest.append(score_xtest)

5it [00:05,  1.07s/it]


In [19]:
print(scores)

[0.8592149294087076, 0.8562532556441094, 0.8671101427100938, 0.8628036875579964, 0.8657239130488963]


In [20]:
print("Mean validation auc: {:.4f} +/- {:.4f}".format(np.mean(scores), np.std(scores)))

Mean validation auc: 0.8622 +/- 0.0040


If we obtain the mean of the scores of all build models during the cross validation, on the testing data, we have:

In [21]:
print("Mean test auc: {:.6f} +/- {:.6f}".format(np.mean(scores_xtest), np.std(scores_xtest))) 

Mean test auc: 0.851781 +/- 0.000184


This is quite similar to what we obtain if we get a mean ensamble of all of the predictions on the testing data and we obtain the score:

In [22]:
mean_preds_test = preds_xtest.mean(axis=1)

In [23]:
roc_auc_score(y_test, mean_preds_test)

0.8518371455673273

Other alternative is to build a model using all the training data and apply it on the testing data (the result in this case is quite similar):

In [24]:
model.fit(X_train, y_train)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=1,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [25]:
full_preds_test = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, full_preds_test)

0.851921935624822

## Cross validation with cross_val_score

In [26]:
cv_scores = cross_val_score(model_lr, X_train, y_train, scoring='roc_auc', cv=kfolds, n_jobs=-1)

In [27]:
cv_scores

array([0.85921493, 0.85625326, 0.86711014, 0.86280369, 0.86572391])

In [28]:
print("Mean validation auc: {:.4f} +/- {:.4f}".format(np.mean(cv_scores), np.std(cv_scores)))

Mean validation auc: 0.8622 +/- 0.0040


## GridSearchCV
This helps also to explore the hyperparameter space

In [29]:
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(class_weight='balanced', solver='lbfgs', random_state=1)
param_grid = {'C': np.logspace(-4, 4, base=10, num=4)}
grid = GridSearchCV(model, cv=kfolds, param_grid=param_grid, n_jobs=-1, scoring='roc_auc')

In [30]:
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=1, test_size='default',
            train_size=None),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=1,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-04, 4.64159e-02, 2.15443e+01, 1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [31]:
grid.best_params_

{'C': 0.0001}

In [32]:
grid.best_score_

0.8622925627423244

In [33]:
cv_results = list(zip(grid.cv_results_['mean_test_score'], 
                      grid.cv_results_['std_test_score']))

for i, param in enumerate(grid.cv_results_['params']):
    print("C: {:.4f} => mean validation auc: {:.4f} +/- {:.4f}"
          .format(param['C'], cv_results[i][0], cv_results[i][1]))

C: 0.0001 => mean validation auc: 0.8623 +/- 0.0040
C: 0.0464 => mean validation auc: 0.8621 +/- 0.0040
C: 21.5443 => mean validation auc: 0.8621 +/- 0.0040
C: 10000.0000 => mean validation auc: 0.8621 +/- 0.0040


In [34]:
grid.best_estimator_

LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=1,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

Logistic Regression score method return the mean accuracy:

In [35]:
grid.best_estimator_.score(X_test, y_test)

0.78155

So, if we want to get the auc metric we can do:

In [36]:
grid.score(X_test, y_test)

0.8526639730751469

The grid object can be used as an estimator because by default the refit param is True. So we can have predicions directly:

In [37]:
(grid.best_estimator_.predict(X_test) != grid.predict(X_test)).sum()

0

## RandomSearchCV
This chooses randomly different n_iter groups of hyperparameters from the defined space and try their performance with cross validation

In [47]:
from sklearn.model_selection import RandomizedSearchCV

model = LogisticRegression(class_weight='balanced', solver='lbfgs', random_state=1)
param_grid = {'C': np.logspace(-4, 4, base=10, num=100)}
grid_random = RandomizedSearchCV(model, cv=kfolds, n_iter=4, 
                                 param_distributions=param_grid, n_jobs=-1, 
                                 random_state=1, scoring='roc_auc')

In [48]:
grid_random.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=1, test_size='default',
            train_size=None),
          error_score='raise-deprecating',
          estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=1,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=4, n_jobs=-1,
          param_distributions={'C': array([1.00000e-04, 1.20450e-04, ..., 8.30218e+03, 1.00000e+04])},
          pre_dispatch='2*n_jobs', random_state=1, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [49]:
grid_random.best_params_

{'C': 0.04641588833612782}

In [52]:
grid_random.best_score_

0.8620898349337865

In [50]:
grid_random.cv_results_['params']

[{'C': 291.5053062825182},
 {'C': 613.5907273413176},
 {'C': 0.04641588833612782},
 {'C': 351.11917342151344}]

In [53]:
cv_results = list(zip(grid_random.cv_results_['mean_test_score'], 
                      grid_random.cv_results_['std_test_score']))

for i, param in enumerate(grid_random.cv_results_['params']):
    print("C: {:.4f} => mean validation auc: {:.4f} +/- {:.4f}"
          .format(param['C'], cv_results[i][0], cv_results[i][1]))

C: 291.5053 => mean validation auc: 0.8621 +/- 0.0040
C: 613.5907 => mean validation auc: 0.8621 +/- 0.0040
C: 0.0464 => mean validation auc: 0.8621 +/- 0.0040
C: 351.1192 => mean validation auc: 0.8621 +/- 0.0040
